In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
from collections import deque
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt
import keypads as ke

In [2]:
def get_data():
    input_text = au.read_txt_file_lines()
    codes = input_text
    print(codes)
    return codes

codes = get_data()
kp = ke.KeyPad()
dr = ke.Direct()

['169A', '279A', '540A', '869A', '789A']


In [3]:
dict_bp = {}
sums = []
n_operators = 2  # number of robots (excl human)
kp = ke.KeyPad()
dr = ke.Direct(shortcuts=True)

for c in codes:
    num_c = int(''.join([x for x in c if x != 'A']))
    c_paths = ke.get_first_keypad_problems(c)
    dict_bp['kp'] = c
    dict_bp[0] = c_paths  ## these are now the instructions for r1.
    print(c_paths)


    ## have to do this twice more for r2 and h
    for i_operator in tqdm(range(1, n_operators + 1)):
        op_paths = []
        for combo in dict_bp[i_operator - 1]:  # previous paths
            tmp_path = []
            combo = 'A' + combo  # start at A position 
            for ii in range(len(combo) - 1):
                d_combo = combo[ii] + combo[ii + 1]
                tmp_path.append(dr.all_paths[d_combo])
            tmp_path = [x if len(x) > 0 else [''] for x in tmp_path]
            tmp_path = list(itertools.product(*tmp_path))
            tmp_path = ke.insert_a_presses(tmp_path)
            for p in tmp_path:
                op_paths.append(p)

        ## greedy but seems to work: only keep shortest paths at each stage. 
        lens = [len(x) for x in op_paths]
        min_len = min(lens)
        lens_keep = [True if l == min_len else False for l in lens]
        # assert sum(lens_keep) == len(lens_keep), (sum(lens_keep), len(lens_keep))
        dict_bp[i_operator] = [x for ix, x in enumerate(op_paths) if lens_keep[ix]]
        # dict_bp[i_operator - 1] = None  # no longer needed
        
    len_c = min([len(x) for x in dict_bp[i_operator]])  # final min len
    print(len_c, num_c)
    print(f'code {c} has {len(dict_bp[i_operator])} combis')
    # print(dict_bp[2])
    sums.append(len_c * num_c)

print(sums)
print(sum(sums))


['^<<A>>^A^AvvvA', '^<<A^>>A^AvvvA']


100%|██████████| 2/2 [00:00<00:00, 16131.94it/s]


76 169
code 169A has 2 combis
['<^A<^^A>>AvvvA', '<^A^^<A>>AvvvA', '^<A<^^A>>AvvvA', '^<A^^<A>>AvvvA']


100%|██████████| 2/2 [00:00<00:00, 19463.13it/s]


72 279
code 279A has 1 combis
['<^^A<A>vvA>A', '^^<A<A>vvA>A']


...

72 540
code 540A has 1 combis
['<^^^Av>A^AvvvA', '<^^^A>vA^AvvvA', '^^^<Av>A^AvvvA', '^^^<A>vA^AvvvA']


100%|██████████| 2/2 [00:00<00:00, 17367.72it/s]


70 869
code 869A has 1 combis
['^^^<<A>A>AvvvA']


100%|██████████| 2/2 [00:00<00:00, 5882.61it/s]

66 789
code 789A has 1 combis
[12844, 20088, 38880, 60830, 52074]
184716


# Idea:
- We know how many (and which) taps keypad `i` needs for keypad `i + 1`. 
- 

In [4]:
def get_data():
    input_text = au.read_txt_file_lines()
    codes = input_text
    print(codes)
    return codes

codes = get_data()
kp = ke.KeyPad()
dr = ke.Direct(shortcuts=False)

key_mapping = {k: ['A'] if len(v) == 0 else [p + 'A' for p in v] for k, v in dr.all_paths.items()}
print(key_mapping)

['169A', '279A', '540A', '869A', '789A']
{'<<': ['A'], '<v': ['>A'], '<>': ['>>A'], '<^': ['>^A'], '<A': ['>>^A', '>^>A'], 'v<': ['<A'], 'vv': ['A'], 'v>': ['>A'], 'v^': ['^A'], 'vA': ['>^A', '^>A'], '><': ['<<A'], '>v': ['<A'], '>>': ['A'], '>^': ['<^A', '^<A'], '>A': ['^A'], '^<': ['v<A'], '^v': ['vA'], '^>': ['v>A', '>vA'], '^^': ['A'], '^A': ['>A'], 'A<': ['v<<A', '<v<A'], 'Av': ['v<A', '<vA'], 'A>': ['vA'], 'A^': ['<A'], 'AA': ['A']}


In [5]:
dict_keypad1_moves_per_keypad = {1: {k: len(v[0]) for k, v in key_mapping.items()}}  # can do v[0] because all elemnts in v are equal (eg <<v, <v<)
## level 1: how many presses does it take the human to move the first robot
## from k[0] to k[1]? Eg for k=<v: 2. (Human needs to >A)

## Now iteratively add another robot. For robot 2, 3, i.. how many human presses 
## are needed for a transition? Once that's known for the final robot,
## we can evaluate the directions for the codepad. 

n_levels = 25
for level in range(2, n_levels + 1):
    dict_keypad1_moves_per_keypad[level] = {} 

    for k, v_list in key_mapping.items():  # check all (1 or 2) combinations (eg A to < can be v<< or <v<)
        best = np.inf
        for v in v_list:
            n_presses = 0
            v_tmp = 'A' + v  # always start at A
            for ii in range(len(v_tmp) - 1):  # eg for k=<^, v=>^A, add transitions >^ and ^A
                n_presses += dict_keypad1_moves_per_keypad[level - 1][v_tmp[ii:ii + 2]]  # get n_presses of previous layer 
            if n_presses < best:
                best = int(n_presses)
        dict_keypad1_moves_per_keypad[level][k] = best

## Compute sum:
grand_total = 0
for c in codes:
    num_c = int(''.join([x for x in c if x != 'A']))
    c_paths = ke.get_first_keypad_problems(c)  # paths on dir keyboard of first robot.
    best = np.inf 
    for cp in c_paths:  # some codes have different paths (of equal length)
        cp = 'A' + cp  # start at A
        total = 0
        for ii in range(len(cp) - 1):
            total += dict_keypad1_moves_per_keypad[level][cp[ii:ii + 2]]  # get numebr of human button presses of this robot 
        if total < best:
            best = total
        print(c, cp, total, num_c)
    grand_total += (int(best) * num_c)
    
print('Total: ', grand_total)

assert grand_total < 479497561257362 and grand_total > 186711883027826, (grand_total / 186711883027826, grand_total / 479497561257362)

169A A^<<A>>^A^AvvvA 92382122304 169
169A A^<<A^>>A^AvvvA 91059074548 169
279A A<^A<^^A>>AvvvA 91387668328 279
279A A<^A^^<A>>AvvvA 94653078920 279
279A A^<A<^^A>>AvvvA 94653078920 279
279A A^<A^^<A>>AvvvA 97918489512 279
540A A<^^A<A>vvA>A 91906526790 540
540A A^^<A<A>vvA>A 95171937382 540
869A A<^^^Av>A^AvvvA 86475783010 869
869A A<^^^A>vA^AvvvA 90750571882 869
869A A^^^<Av>A^AvvvA 89741193602 869
869A A^^^<A>vA^AvvvA 94015982474 869
789A A^^^<<A>A>AvvvA 80786362260 789
Total:  229403562787554
